<a href="https://colab.research.google.com/github/guilledd10/SEMINARIO/blob/main/Seminario_Algoritmos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Seminario<br>
Nombre y Apellidos:   Guillermo Dávila Duarte
Url: https://github.com/guilledd10/SEMINARIO/blob/main/Seminario_Algoritmos.ipynb<br>
Problema:
>3. Combinar cifras y operaciones

Descripción del problema:



*   El problema consiste en analizar el siguiente problema y diseñar un algoritmo que lo resuelva.
*   Disponemos de las 9 cifras del 1 al 9 (excluimos el cero) y de los 4 signos básicos de las operaciones fundamentales: suma(+), resta(-), multiplicación(*) y división(/)
*   Debemos combinarlos alternativamente sin repetir ninguno de ellos para obtener una cantidad dada. Un ejemplo sería para obtener el 4: 4+2-6/3*1 = 4


*   Debe analizarse el problema para encontrar todos los valores enteros posibles planteando las
siguientes cuestiones:
1.   ¿Qué valor máximo y mínimo se pueden obtener según las condiciones del problema?
2.   Elemento de lista



*   Nota: Es posible usar la función de python “eval” para evaluar una expresión:

In [1]:
expresion = "4+2-6/3*1"
print(eval(expresion))

4.0


(*)¿Cuantas posibilidades hay sin tener en cuenta las restricciones?<br>



¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones.




Respuesta

*   Número de combinaciones sin restriciones:
1.   5 posiciones de dígitos (con repetición): 9^5
2.   4 posiciones de operadores (con repetición): 4^4
Total:

In [2]:
9 ** 5 * 4 ** 4

15116544

*   Número de combinaciones con restriciones:


1.   Permutaciones de 5 dígitos distintos de 9: P(9,5) = 9 * 8 * 7 * 6 * 5
2.   Permutaciones de los 4 operadores: 4!

Total:

In [3]:
import math
9 * 8 * 7 * 6 * 5 * math.factorial(4)

362880

Modelo para el espacio de soluciones<br>
(*) ¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguentalo)


Respuesta

*   itertools.permutations para generar digitos y operadores sin restriciones.
*   set() para almacenar resultados únicos de enteros.
*   dict() opcional para mapear valor

Según el modelo para el espacio de soluciones<br>
(*)¿Cual es la función objetivo?

(*)¿Es un problema de maximización o minimización?

Respuesta



*   Para la pregunta de buscar extremos debemos maximizar y minimizar el valor de la expresión, según estemos buscando un máximo o un mínimo.
*  En el caso de alcanzar un número objetivo T: hay que minimizar |resultado - T|



Diseña un algoritmo para resolver el problema por fuerza bruta

Respuesta

Este algoritmo genera todas las expresiones posibles y evalúa cada una usando **eval**. Se filtran los resultados que sean enteros.

In [4]:
import itertools
from math import isclose

def fuerza_bruta():
    digits = list(range(1, 10))
    ops = ['+', '-', '*', '/']
    resultados = set()
    exprs = {}

    for nums in itertools.permutations(digits, 5):
        for ops_order in itertools.permutations(ops):
            expr = "".join(str(nums[i]) + ops_order[i] for i in range(4)) + str(nums[4])
            try:
                val = eval(expr)
                if isclose(val, round(val), abs_tol=1e-9):
                    ival = int(round(val))
                    resultados.add(ival)
                    exprs.setdefault(ival, expr)
            except ZeroDivisionError:
                continue
    return resultados, exprs

Calcula la complejidad del algoritmo por fuerza bruta

Respuesta

Es el orden que se ha visto antes en las posibilidades con restricciones: **O(362.880)**

(*)Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta

Respuesta

El algoritmo seleccionado es con poda (backtracking), en este caso evita las ramas que dan resultados no válidos, por ejemplo aquellos resultados en los que se detecta división no exacta o valores fuera de un rango normal.

In [5]:
def optimizado_poda():
    digits = list(range(1, 10))
    ops = ['+', '-', '*', '/']
    resultados = set()

    def backtrack(nums_rest, ops_rest, expr):
        if not nums_rest and not ops_rest:
            try:
                val = eval(expr)
                if isclose(val, round(val), abs_tol=1e-9):
                    resultados.add(int(round(val)))
            except ZeroDivisionError:
                return
            return

        for i, d in enumerate(nums_rest):
            for j, op in enumerate(ops_rest):
                new_expr = expr + op + str(d)
                try:
                    val = eval(new_expr)
                except ZeroDivisionError:
                    continue
                if abs(val) > 1000:  # poda simple
                    continue
                backtrack(
                    nums_rest[:i] + nums_rest[i+1:],
                    ops_rest[:j] + ops_rest[j+1:],
                    new_expr
                )

    for i, d in enumerate(digits):
        backtrack(digits[:i] + digits[i+1:], ops, str(d))

    return resultados

(*)Calcula la complejidad del algoritmo

Respuesta

La complejidad depende de las operaciones y números, pudiendo ser el peor caso teórico el del algoritmo por fuerza voraz.

Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios

Respuesta

Aplica el algoritmo al juego de datos generado

Respuesta

In [6]:
# Ejecutar fuerza bruta
res, exprs = fuerza_bruta()

# Mostrar dos expresiones ejemplo
import random
ejemplos = random.sample(list(exprs.items()), 2)
for val, ex in ejemplos:
    print(f"Ejemplo: {ex} = {val}")

# Comprobar el ejemplo dado en el enunciado
ejemplo_expr = "4+2-6/3*1"
print("Expresión ejemplo:", ejemplo_expr, "Resultado:", eval(ejemplo_expr))

# Métricas finales
min_val, max_val = min(res), max(res)
cobertura = all(v in res for v in range(min_val, max_val+1))
print("Valor mínimo:", min_val, "Valor máximo:", max_val, "Cobertura total:", cobertura)

Ejemplo: 1-5*8+6/2 = -36
Ejemplo: 1-4*9+6/3 = -33
Expresión ejemplo: 4+2-6/3*1 Resultado: 4.0
Valor mínimo: -69 Valor máximo: 77 Cobertura total: True


Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo

Respuesta

Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño

Se podrían usar otros algoritmos como:


*   Branch and Bound: explora el árbol de combinaciones y poda ramas cuando no pueden mejorar la solución.
*   Programación dinámica: reutiliza resultados de subexpresiones para evitar cálculos repertidos.
